#### Laboratorio 9
Stefano Aragoni, Luis Diego Santos, Carol Arevalo

1. Variables Crisp

Distancia entre el robot y la pelota.

2. Variables Lingüísticas:
Cerca, Medio, Lejos.


3. Cláusulas de Horn para encontrar la pelota:

IF distancia es CERCA THEN movimiento es LENTO
IF distancia es MEDIO THEN movimiento es MODERADO
IF distancia es LEJOS THEN movimiento es RÁPIDO


4. Defuzzificación para encontrar la pelota:
Vamos a usar el método del centroide.


5. Variables Crisp para saber la fuerza con la que se debe patear la pelota:
Distancia entre la pelota y la portería.

6. Variables Lingüísticas para saber la fuerza con la que se debe patear la pelota:
Débil, Medio, Fuerte.


7. Cláusulas de Horn para saber la fuerza con la que se debe patear la pelota:
IF distancia a la portería es CERCA THEN fuerza es DÉBIL
IF distancia a la portería es MEDIO THEN fuerza es MEDIO
IF distancia a la portería es LEJOS THEN fuerza es FUERTE


8. Defuzzificación para saber la fuerza con la que se debe patear la pelota:
Nuevamente, usaremos el método del centroide.


9. Grafique las funciones de pertenencia para encontrar la pelota

10. Grafique las funciones de pertenencia para saber la fuerza con la que se debe patear la pelota

11. Programe la interfaz gráfica donde se pueda seguir el movimiento del robot y la pelota

In [2]:
import pygame
import numpy as np

# Inicialización de Pygame
pygame.init()

# Dimensiones de la ventana
WIDTH, HEIGHT = 800, 600

# Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 128, 0)  # Modificado para representar el color del campo de juego
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)

# Ventana y reloj
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Robot Soccer")
clock = pygame.time.Clock()

class Robot:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.speed = 1

    def move_towards(self, target_x, target_y):
        dist = np.sqrt((target_x - self.x)**2 + (target_y - self.y)**2)
        
        # Lógica difusa para la velocidad
        if dist < 100:
            self.speed = 1  # cerca
        elif dist < 300:
            self.speed = 3  # medio
        else:
            self.speed = 5  # lejos

        direction = (target_x - self.x, target_y - self.y)
        norm = np.sqrt(direction[0]**2 + direction[1]**2)
        direction = (direction[0] / norm, direction[1] / norm)

        self.x += direction[0] * self.speed
        self.y += direction[1] * self.speed

    def draw(self, win):
        pygame.draw.rect(win, BLACK, (self.x-10, self.y-20, 20, 40))  # Robot representado como un rectángulo

class Ball:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def move_towards(self, target_x, target_y, speed=4):
        direction = (target_x - self.x, target_y - self.y)
        norm = np.sqrt(direction[0]**2 + direction[1]**2)
        
        # Umbral de llegada
        if norm < 10:
            return

        direction = (direction[0] / norm, direction[1] / norm)

        self.x += direction[0] * speed
        self.y += direction[1] * speed

    def draw(self, win):
        pygame.draw.circle(win, WHITE, (int(self.x), int(self.y)), 15)

robot = Robot(200, HEIGHT // 2)
ball = Ball(400, HEIGHT // 2)

def redraw_window():
    # Dibuja el campo de fútbol
    win.fill(GREEN)
    pygame.draw.rect(win, WHITE, (WIDTH-150, HEIGHT//4, 5, HEIGHT//2))
    pygame.draw.circle(win, WHITE, (WIDTH//2, HEIGHT//2), 150, 5)  # Círculo central
    robot.draw(win)
    ball.draw(win)
    pygame.display.update()


def main():
    ball_kicked = False
    run = True
    while run:
        clock.tick(30)  # Limitamos a 30 FPS
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        
        distance_to_ball = np.sqrt((robot.x - ball.x)**2 + (robot.y - ball.y)**2)
        
        # Si el robot está lo suficientemente cerca de la pelota, la pelota se mueve hacia la portería
        if distance_to_ball < 50 and not ball_kicked:
            ball_kicked = True
        
        if ball_kicked:
            ball.move_towards(WIDTH-150, HEIGHT//2)
        else:
            robot.move_towards(ball.x, ball.y)
        
        redraw_window()

    pygame.quit()

if __name__ == "__main__":
    main()
